In [20]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords

In [21]:
df=pd.read_csv('fdata.csv')

In [22]:
df.head()


,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5842 entries, 0 to 5841
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Sentence   5842 non-null   object
 1   Sentiment  5842 non-null   object
dtypes: object(2)
memory usage: 91.4+ KB


In [24]:
df.describe()

,Sentence,Sentiment
count,5842,5842
unique,5322,3
top,Managing Director 's comments : `` Net sales f...,neutral
freq,2,3130


In [25]:
import string
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
def clean_text(text):
    # create a list of stop words
    stop_words = [stopwords.words("english")]
    
    no_punc = [word for word in text if word not in string.punctuation]
    
    no_punc_str = "".join(no_punc)
    
    return "".join([word for word in no_punc_str if word.lower() not in stop_words])

In [27]:
df["cleaned"] = df["Sentence"].apply(clean_text)

In [28]:
df.head()

,Sentence,Sentiment,cleaned
0,The GeoSolutions technology will leverage Bene...,positive,The GeoSolutions technology will leverage Bene...
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative,ESI on lows down 150 to 250 BK a real possibility
2,"For the last quarter of 2010 , Componenta 's n...",positive,For the last quarter of 2010 Componenta s net...
3,According to the Finnish-Russian Chamber of Co...,neutral,According to the FinnishRussian Chamber of Com...
4,The Swedish buyout firm has sold its remaining...,neutral,The Swedish buyout firm has sold its remaining...


In [29]:
df["mapped_sentiments"] = np.where(df["Sentiment"]=="positive", 1, np.where(df["Sentiment"]=="negative", -1, 0))

In [30]:
df.head()

,Sentence,Sentiment,cleaned,mapped_sentiments
0,The GeoSolutions technology will leverage Bene...,positive,The GeoSolutions technology will leverage Bene...,1
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative,ESI on lows down 150 to 250 BK a real possibility,-1
2,"For the last quarter of 2010 , Componenta 's n...",positive,For the last quarter of 2010 Componenta s net...,1
3,According to the Finnish-Russian Chamber of Co...,neutral,According to the FinnishRussian Chamber of Com...,0
4,The Swedish buyout firm has sold its remaining...,neutral,The Swedish buyout firm has sold its remaining...,0


In [31]:
df = df.drop(["Sentence", "Sentiment"], axis=1)

In [32]:
df.head()

,cleaned,mapped_sentiments
0,The GeoSolutions technology will leverage Bene...,1
1,ESI on lows down 150 to 250 BK a real possibility,-1
2,For the last quarter of 2010 Componenta s net...,1
3,According to the FinnishRussian Chamber of Com...,0
4,The Swedish buyout firm has sold its remaining...,0


In [33]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

In [34]:
X = df["cleaned"]

y = df["mapped_sentiments"]

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [36]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(4381,) (4381,)
(1461,) (1461,)


In [37]:
count_vec = CountVectorizer()
# Fit on train set
count_vec.fit(X_train)
# Transform the train and test sets

X_train_vec = count_vec.transform(X_train)
X_test_vec = count_vec.transform(X_test)

In [38]:
from sklearn.feature_extraction.text import TfidfTransformer

In [39]:
tfidf = TfidfTransformer()

tfidf.fit(X_train_vec)
tfidf.transform(X_train_vec)

<4381x10899 sparse matrix of type '<class 'numpy.float64'>'
	with 71986 stored elements in Compressed Sparse Row format>

In [40]:
model = MultinomialNB()

In [41]:
X_train_vec.shape, X_test_vec.shape, y_train.shape, y_test.shape

((4381, 10899), (1461, 10899), (4381,), (1461,))

In [42]:
model.fit(X_train_vec, y_train)

MultinomialNB()

In [46]:
pred = model.predict(X_test_vec)
pred

array([ 0,  0, -1, ...,  0,  0,  0])

In [44]:
accuracy_score(y_test, pred)

0.6988364134154689

In [45]:
X_test_vec

<1461x10899 sparse matrix of type '<class 'numpy.int64'>'
	with 21770 stored elements in Compressed Sparse Row format>

In [53]:
testnews = ["Banks tumble as SVB ignites capitalization fears"]
# clean_test_news = clean_text(testnews)
clean_test_news1 = count_vec.transform(testnews)
pred1 = model.predict(clean_test_news1)
pred1

array([-1])

In [55]:
type(pred1)

numpy.ndarray

In [59]:
pred1[0]

-1

In [62]:
#Business Logic

if (pred1[0] == 1):
    print("News is Positive")
elif (pred1[0] == -1):
    print("News is Negative")
else:
    print("News is Neutral")

News is Negative
